# Sentinel Functions

With this module we can scale the values of SR image from Sentinel-2 image collection and mask their clouds

In [1]:
import ee
from IPython.display import Image
from statgis.gee.sentinel_functions import cloud_mask, scaler

ee.Initialize()

Define the region of interest

In [2]:
coords = [-73.452, -37.093, -72.727, -36.525]

roi = ee.Geometry.BBox(*coords)
roi.area(10).getInfo()

4076535807.291905

We will mask the clouds and scale the values to SR in all images from Landsat 8 in the region of interest

In [3]:
img = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(roi)
      .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 20))
      .map(cloud_mask)
      .map(scaler)
      .median()
)

# Show all reflectance bands
[band["id"] for band in img.getInfo()["bands"] if band["id"][0] == "B"]

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

Now we can see the results

In [4]:
vis = {
    "bands": [f"B{i}" for i in "432"],
    "min": 0.0,
    "max": 0.3,
    "region": roi,
    "dimensions": 512,
}

url = img.getThumbURL(vis)
Image(url=url)